Neural Model Identification: \
Trains a neural network based on data from the physics model to use as the predictive model
Note: You need to install the matlab engine to be able to run matlab code from pythone

In [1]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt

In [2]:
import os
import pickle
import math
#from utils import kits
import time
import random 
import numpy as np
import tensorflow as tf
import scipy.io
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Activation
from tensorflow.keras import backend as K
from tensorflow.python.framework import ops
from sklearn import preprocessing


In [3]:
class neural_model:
    def __init__(self,epochs,bs):
        """
        epochs:Number of epochs, should be over 10^2 for training from scratch, a few for TL
        bs: bs<Ndata in general but bs=Ndata during TL
        """
        get_custom_objects().update({'swish': Activation(neural_model.swish)})
        self.lr=0.1
        self.layers = 6
        self.nodes = 12
        self.bs=bs
        self.epochs=epochs
        self.activation="swish"
    
    def swish(x, beta=1):
        return x * sigmoid(beta * x)

    def compile_model(nodes, layers, activation, learning_rate,in_dim, out_dim, trainable, training=False):
        if activation == 'swish':
            activation = neural_model.swish
        else:
            activation = activation

        inputs = tf.keras.layers.Input(shape=(in_dim,),)
        x = tf.keras.layers.Dense(units=nodes, activation=activation,trainable=trainable)(inputs)
        for i in range(layers - 2):  # at least one layer which has the parameter input as input
            x = tf.keras.layers.Dense(units=nodes, activation=activation,trainable=trainable)(x)
            #x.trainable = False
        x = tf.keras.layers.Dense(3, activation=activation,trainable=trainable)(x)       
       
        outputs = tf.keras.layers.Dense(out_dim, activation="linear", trainable=True)(x)
        outputs = tf.keras.layers.Add()([outputs, inputs[:, 0:3]]) #Residual Layer
        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        optimizer = optimizers.Adagrad(lr=learning_rate)# amsgrad=True)
        model.compile(optimizer=optimizer,
                      loss='mean_absolute_error')
        return model
   
    def get_model(self, X_train, Y_train, pre_train, weights=[], X_test=[],Y_test=[], validation=False):
     
        """
        X_train, Y_train: Features and labels to be trained on
        pre_train: True is pretrained weights are available, False otherwsie (Boolean)
        X_test, Y_test: Features and labels for the testing set 
        """
        #print(self.lr,
        #self.layers,
        #self.nodes,
        #self.bs,
        #self.epochs,
        #self.activation)
        
        in_dim = X_train.shape[1]
        out_dim = Y_train.shape[1]
        
        nnet = neural_model.compile_model(nodes=self.nodes, layers=self.layers,
                             activation=self.activation, learning_rate=self.lr,in_dim=in_dim,out_dim=out_dim,
                             trainable=(not pre_train), training=False)  # Training true-activates dropout during prediction
        if (pre_train):
            nnet.set_weights(weights)
        
        nnet.summary()        
        
        if (validation):
            nnet.fit(X_train, Y_train,
                      batch_size=self.bs,
                      epochs=self.epochs,
                      verbose=1,
                      validation_data = (X_test, Y_test))
        else:
            nnet.fit(X_train, Y_train,
                     batch_size=self.bs,
                      epochs=self.epochs,
                      verbose=1)
            
            
        return nnet 

